In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
# 0) Prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

In [4]:
n_samples, n_features = X.shape
print(n_samples, n_features)

569 30


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

In [6]:
# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [8]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [9]:
# 1) Model
# Linear model f = wx + b , sigmoid at the end
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [10]:
# 1) Model
# Linear model f = wx + b
model = Model(n_features)
model

Model(
  (linear): Linear(in_features=30, out_features=1, bias=True)
)

In [11]:
# 2) Loss and optimizer
learning_rate = 0.01
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [13]:
# 3) Training loop
num_epochs = 500
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward pass and update
    loss.backward()
    optimizer.step() # update the weights

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 50 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 50, loss = 0.2090
epoch: 100, loss = 0.1818
epoch: 150, loss = 0.1632
epoch: 200, loss = 0.1494
epoch: 250, loss = 0.1388
epoch: 300, loss = 0.1303
epoch: 350, loss = 0.1234
epoch: 400, loss = 0.1176
epoch: 450, loss = 0.1126
epoch: 500, loss = 0.1084


In [14]:
# 4) Prediction
with torch.no_grad():  # not to calculate gradient
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.9386
